In [ ]:
!pip install -q transformers datasets torch scikit-learn pandas fastapi uvicorn pyngrok


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import sqlite3
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)


In [ ]:
import json
import pandas as pd


data = []
file_path = "/News_Category_Dataset.json"
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError:
            # Silently skip malformed lines to prevent the program from crashing.
            pass

df = pd.DataFrame(data)

print("Dataset shape:", df.shape)
df.head()

**Data** **Preparation**

In [ ]:
# Combine headline and description
df["text"] = df["headline"] + " " + df["short_description"]

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["category"])

In [ ]:
num_labels = len(label_encoder.classes_)
print("Number of classes:", num_labels)

In [ ]:
# Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

**2 Model Selection**
distilbert-base-uncased

**3 Tokenization**

In [ ]:
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)


In [ ]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }


**4 Training**

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # tokenizer=tokenizer, # Removed this argument as it's no longer accepted
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
import matplotlib.pyplot as plt

epochs = [1, 2, 3]
train_loss = [1.633, 1.099, 0.918]
val_loss = [1.153, 1.005, 0.972]
accuracy = [0.692, 0.719, 0.731]
f1 = [0.638, 0.685, 0.704]

plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, accuracy, label='Accuracy')
plt.plot(epochs, f1, label='Weighted F1')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.show()


**5 Evaluation **

In [ ]:
eval_results = trainer.evaluate()
eval_results


In [ ]:
#Save Model & Labels
model.save_pretrained("model")
tokenizer.save_pretrained("model")

with open("labels.json", "w") as f:
    json.dump(label_encoder.classes_.tolist(), f)

print("Model and labels saved.")


**SQLite Database Setup**

In [ ]:
conn = sqlite3.connect("predictions.db", check_same_thread=False)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    input_text TEXT,
    predicted_category TEXT,
    confidence REAL,
    created_at TEXT
)
""")

conn.commit()
print("Database initialized.")


In [ ]:
#Inference Function
with open("labels.json") as f:
    labels = json.load(f)

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    confidence, predicted_idx = torch.max(probs, dim=1)

    return labels[predicted_idx.item()], confidence.item()


**FastAPI Backend**

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="News Category Classifier")

class InputText(BaseModel):
    text: str

@app.post("/predict")
def predict_category(request: InputText):
    category, confidence = predict(request.text)

    cursor.execute(
        "INSERT INTO predictions VALUES (NULL, ?, ?, ?, ?)",
        (request.text, category, confidence, datetime.utcnow().isoformat())
    )
    conn.commit()

    return {
        "predicted_category": category,
        "confidence": round(confidence, 2)
    }


To make the following public url accessing work, kindly create the ngrok authentication token and replace "YOUR_NGROK_AUTH_TOKEN" and "NGROK_AUTH_TOKEN"

In [ ]:
# from pyngrok import ngrok
# import uvicorn
# import os

# os.environ["NGROK_AUTH_TOKEN"] = "YOUR_NGROK_AUTH_TOKEN"

# ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

# ngrok.kill()

# public_url = ngrok.connect(8000)
# print("Public URL:", public_url)

# uvicorn.run(app, host="0.0.0.0", port=8000)
